In [1]:
import pandas as pd
from geopy.distance import distance
from config_bd import get_connection
from credenciales_sqlserver import *

In [2]:
completo = pd.read_csv('transform_listings.csv')

In [3]:
with get_connection() as conn:
    query = """
    SELECT * FROM TRANSPORTS
    """
    stops = pd.read_sql(query, conn)
    

C:\Users\dsantamaria\AppData\Local\Temp\ipykernel_22828\720485261.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stops = pd.read_sql(query, conn)


In [4]:
stops

,id,mode,stop_name,stop_desc,lat,lon
0,1423,4,PLAZA DE CASTILLA,Paseo de la Castellana 189,40.46690,-3.68917
1,1424,4,TRIBUNAL,Calle de Fuencarral 81,40.42619,-3.70110
2,1425,4,GRAN VIA,Calle Gran Vía 23,40.42001,-3.70180
3,1426,4,SOL,Plaza de la Puerta del Sol 6,40.41688,-3.70326
4,1427,4,TIRSO DE MOLINA,Plaza de Tirso de Molina 7,40.41235,-3.70466
...,...,...,...,...,...,...
13648,15071,10,ESTRELLA POLAR NORTE,Avda de las Estrellas 14,40.23015,-3.75112
13649,15072,10,ESTRELLA POLAR SUR,Avda de los Planetas 9,40.23072,-3.75383
13650,15073,10,JAIME I NORTE,Avda de las Estrellas 2,40.23414,-3.74969
13651,15074,10,JAIME I SUR,Avda de los Planetas 6,40.23470,-3.75240


In [ ]:
TIPOS_TRANSPORTE = [4,5,6,8,10] #El 9 está integrado en el 8

In [ ]:
def calcular_distancias_vivienda(row_viv):
    resultados = {}
    
    for modo in TIPOS_TRANSPORTE:
        paradas_tipo = stops[stops['mode'] == modo]

        if paradas_tipo.empty:
            resultados[f'distancia_mode_{modo}'] = None
            continue

        # filtrar primero por lat/lon cercanas para acelerar
        delta = 0.03 #unos 8km cuadrados de margen
        paradas_candidatas = paradas_tipo[
            (abs(paradas_tipo['lat'] - row_viv['LATITUDE']) < delta) &
            (abs(paradas_tipo['lon'] - row_viv['LONGITUDE']) < delta)
        ]

        if paradas_candidatas.empty:
            paradas_candidatas = paradas_tipo  # fallback a todas

        distancias = paradas_candidatas.apply(
            lambda stop: distance(
                (row_viv['LATITUDE'], row_viv['LONGITUDE']),
                (stop['lat'], stop['lon'])
            ).meters,
            axis=1
        )

        resultados[f'distancia_mode_{modo}'] = distancias.min()

    return pd.Series(resultados)



In [ ]:

# df = df.sample(200)
df_distancias = df.apply(calcular_distancias_vivienda, axis=1)

df = pd.concat([df, df_distancias], axis=1)
df